In [17]:
import os, csv, json, requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import clear_output

from pathlib import Path
pd.options.display.float_format = '{:.2f}'.format

In [3]:
home = Path(os.getcwd())
xls_file = home.joinpath('SLE_results1.13.xlsx')
csv_outfile = home.joinpath('SLE_results1.13_PVGIS.csv')


### pandas

In [24]:
df_xls = pd.read_excel(xls_file)

columns = ['idSet','latitude','longitude']
# df = df_xls[columns].copy()

df = df_xls[columns].head(2).copy() # test only

print(df)

   idSet  latitude  longitude
0      1      7.82     -10.92
1      2      8.28     -10.38


### create urls

In [25]:
def get_url(lat, lon):    
    
    url_base = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?"
    
    pvgis_params = dict(
        peakpower=1,
        loss=14,
        startyear=2010,
        endyear=2020,
        pvcalculation=1, # "0" outputs only solar radiation calculations, "1" outputs the estimation of hourly PV production as well
        angle = 90,
        aspect = 90,
        lat=lat,
        lon=lon,
        outputformat = 'json',
    )

    return url_base + "&".join([f'{key}={value}' for key, value in pvgis_params.items()])


In [26]:
def get_azimuth_from_latitude(lat):

    if float(lat) < 0:
        azimuth = 180
    else:
        azimuth=0

    return azimuth


In [32]:
def parse_json(id, data, csv_file):    
    
     # ____________ parse data to df
    df_locations    = pd.json_normalize(data.get('inputs'))
    df_output       = pd.json_normalize(data.get('outputs').get('hourly'))    

    df_power         = df_output[['time', 'P']].copy() 
    df_power['time'] = pd.to_datetime(df_power['time'], format='%Y%m%d:%H%M').round('H').dt.strftime('%Y-%m-%d:%H')
    
    print (df_power.head())
    
    # ____________ transpose df
    # df_tranpose = df_power[['P']].transpose()    
    
    
    df_tranpose = df_power.set_index('time').T
    
    # ____________ add id
    df_tranpose.insert(0, 'id', id)
    
    print (df_tranpose.head())
    
    return df_tranpose

    
#     # _____________ parse data from pvgis
#     pvgis_inputs  = data.get('inputs')
#     pvgis_outputs = data.get('outputs')
    
#     # _____________ get specific parameters data from pvgis
#     location = pvgis_inputs.get('location')
#     latitude = location.get('latitude')
#     longitude = location.get('longitude')
    
#     slope_optimized       = pvgis_inputs.get('mounting_system').get('fixed').get('slope').get('value')
#     azimuth_non_optimized = pvgis_inputs.get('mounting_system').get('fixed').get('azimuth').get('value')
#     azimuth_corrected     = get_azimuth_from_latitude(latitude)
    

#     # _____________ create a new dict with filtered json data
#     inputs = location
#     outputs = dict(hourly = pvgis_outputs.get('hourly'))
#     parameters = dict(
#             id=id, 
#             slope_optimized=slope_optimized, 
#             azimuth_non_optimized=azimuth_non_optimized,
#             azimuth_corrected=azimuth_corrected,
#             # url = url
#         )      
    
#     # _____________ merge dicts         
#     merged_dict = dict(inputs = inputs, outputs=outputs, parameters=parameters)
    
#     print (merged_dict)
    
#     # _____________ save json
#     with open(json_file, 'w') as outfile:
#         json.dump(merged_dict, outfile)

    
    

### query PVGIS

In [33]:
i = 0
dfs = dict()

for idx, row in df.iterrows(): 
    
    id = row['idSet'].astype(int)
    lat = row['latitude'].astype(str)
    lon = row['longitude'].astype(str)
    url = get_url (lat,lon)
    
    csv_file = home.joinpath('csv', f"SLE_{id}.csv")
    
    print (url)

    response = requests.get(url)
    row_json = json.loads(response.text)    

    # try:    
        # _____________ parse data from pvgis
    dfs[id] = parse_json(id, row_json, csv_file)
        # e_y = row_json.get('outputs').get('totals').get('fixed').get('E_y')
        # print (f'i:{i}\tid: {id}\tside: \tradiation: {e_y}')
#     except:
        
#         print ('error')
#         # e_y = -9999
#         # print (f'i:{i}\t\t\tERROR\nid: {id}\tside: \tradiation: {e_y}\t\tlat: {lat}\tlon: {lon}')            

    # _____________ update df
    # df.loc[df["idSet"] == id, "E_Y"] = e_y

#     # _____________ save partial results
#     if i > 100:  
#         print ('_____________________________')
#         df.to_csv(csv_outfile, index = False)
#         i = 0
#         clear_output(wait=False)

#     i +=1
            

# print (df.head())
# _____________ save final results

df_merged = pd.concat(dfs.values(), ignore_index=True)

df_merged.to_csv(csv_outfile, index = False)

https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?peakpower=1&loss=14&startyear=2010&endyear=2020&pvcalculation=1&angle=90&aspect=90&lat=7.822542&lon=-10.92325&outputformat=json
            time    P
0  2010-01-01:00 0.00
1  2010-01-01:01 0.00
2  2010-01-01:02 0.00
3  2010-01-01:03 0.00
4  2010-01-01:04 0.00
   id    0    1    2    3    4    5    6    7     8  ...  96422  96423  96424  \
P   1 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 25.95  ... 312.49 437.39 490.31   

   96425  96426  96427  96428  96429  96430  96431  
P 424.86 159.47   0.00   0.00   0.00   0.00   0.00  

[1 rows x 96433 columns]
https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?peakpower=1&loss=14&startyear=2010&endyear=2020&pvcalculation=1&angle=90&aspect=90&lat=8.282627&lon=-10.37537&outputformat=json
            time    P
0  2010-01-01:00 0.00
1  2010-01-01:01 0.00
2  2010-01-01:02 0.00
3  2010-01-01:03 0.00
4  2010-01-01:04 0.00
   id    0    1    2    3    4    5    6    7     8  ...  96422  96423  96424  \
P   2 0.